In [1]:
from functools import reduce
import offsb.chem.types

Because hydrogen really only have one bit of information (for our purposes it is always `[#1H0X1x0!rA]`), we turn on an experimental setting which should handle hydrogen as a special case, which has implications for chemical space, bit manipulations, and bit iterating.

In [2]:
offsb.chem.types.HYDROGEN_ONE_BIT = True

### Load the exact same CCH angle SMARTS, but reversed

In [3]:
x = offsb.chem.types.AngleGraph.from_smarts("[#6H1X4x0!r+0A]-;!@[#6H3X4x0!r+0A]-;!@[#1]")
y = offsb.chem.types.AngleGraph.from_smarts("[#1]-;!@[#6H3X4x0!r+0A]-;!@[#6H1X4x0!r+0A]")

In [4]:
x

(S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10)

In [5]:
x.to_smarts()

'[#6;H1;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#1:3]'

In [6]:
y

(S:  10) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1)

In [7]:
y.to_smarts()

'[#1:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#6;H1;X4;x0;!r;A:3]'

Equality uses symmetry information, so it will check if ABC == {DEG or GED} 

In [8]:
x == y

True

Other operations, however, do not use any symmetry. Here we add `x` and `y` together with no shuffling: A + D, B + E, C + G. Note that addition is equivalent to union `x & y`

In [9]:
xy = x + y ; xy

(S:  1000010 H:  10 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000010 H:  10 X:  10000 x:   1 r:   1 aA:   1)

A subtle caveat to this is now the angle encodes HCH, and CCC angles

In [10]:
xy.to_smarts()

'[#6,#1;H1;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#6,#1;H1;X4;x0;!r;A:3]'

In [11]:
for i, primitive in enumerate(xy.to_primitives()):
    print(f"{i+1:2d}", primitive.to_smarts())

 1 [#1:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#1:3]
 2 [#6;H1;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#1:3]
 3 [#6;H1;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#6;H1;X4;x0;!r;A:3]


In [12]:
x == xy

False

In [13]:
x in xy

True

In [14]:
y in xy

True

In [15]:
(x - y).bits()

7

In [16]:
x-y

(S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1) [Order:   0 aA:   0] (S:   0 H:   0 X:   0 x:   0 r:   0 aA:   0) [Order:   0 aA:   0] (S:  10)

We can align the angles, meaning it tries to overlap as many bits as possible. It modifies `x` in place.

In [17]:
x.align_to(y) ; x

(S:  10) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1)

In [18]:
(x - y).bits()

0

In [19]:
x == x + y

True

In [20]:
x ^ y

(S:   0) [Order:   0 aA:   0] (S:   0 H:   0 X:   0 x:   0 r:   0 aA:   0) [Order:   0 aA:   0] (S:   0 H:   0 X:   0 x:   0 r:   0 aA:   0)

### Patterns with multiple bits

Since `a` is unbounded in H (any H value is accepted), we see that it contains an unbounded number of bits

In [21]:
a = offsb.chem.types.AngleGraph.from_smarts("[#6X4x0!r+0A]-;!@[#6H3X4x0!r+0A]-;!@[#1]")
a.bits()

inf

All fields have a default `maxbits` set, so we know roughly how many bits in each field are actually useful. Here, for H, `maxbits=5`, corresponding to H0-H4.

In [22]:
a.bits(maxbits=True)

21

In [23]:
print(a)
print(x)

(S:  1000000 H: ~ 0 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10)
(S:  10) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1)


In [24]:
x.align_to(a)
print(a)
print(x)

(S:  1000000 H: ~ 0 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10)
(S:  1000000 H:  10 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10)


In [25]:
a - x

(S:   0 H: ~10 X:   0 x:   0 r:   0 aA:   0) [Order:   0 aA:   0] (S:   0 H:   0 X:   0 x:   0 r:   0 aA:   0) [Order:   0 aA:   0] (S:   0)

The `~10` means it inverts the pattern, so instead of `...0000000010`, with infinite leading zeros, it has infinite leading 1s: `...111111101`.
This means `a` has info beyond `x` on all H values except H1.

This shows that bits can be directly accessed. Since `~0` signifies an unbounded bit vector, we see that accessing the 4th bit returns `True`

In [26]:
c = a.copy()
c._atom1._H[3]

True

Slices also work

In [27]:
c._atom1._H[:6]

array([ True,  True,  True,  True,  True,  True])

as do assignments

In [28]:
c._atom1._H[2:4] = False ; c

(S:  1000000 H: ~1100 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10 H:   0 X:   0 x:   0 r:   0 aA:   0)

We can also indicate the behavior of the leading bit by using an unbounded slice.

In [29]:
c._atom1._H[5:] = False ; c

(S:  1000000 H:  10011 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10 H:   0 X:   0 x:   0 r:   0 aA:   0)

There should never be index errors, as the vectors will automatically grow to fit the desired length. These are explicit bit vectors, so all values are stored up until the point where we know the unbounded value

In [30]:
bv = offsb.chem.types.BitVec()
print(bv, bv.any(), bv.all())

bv[100] = True
print(bv, bv.any(), bv.all())

bv = ~bv
print(bv, bv.any(), bv.all())

bv[10] = True
print(bv, bv.any(), bv.all())

bv[10:30:2] = False
print(bv, bv.any(), bv.all())

bv[40:] = False
print(bv, bv.any(), bv.all())

bv[:] = True
print(bv, bv.any(), bv.all())

  0 False False
 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 True False
~10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 True False
~10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 True False
~10000000000000000000000000000000000000000000000000000000000000000000000010101010101010101010000000000 True False
 1111111111101010101010101010101111111111 True False
~ 0 True True


Iterating the bits will always use `maxbits`, meaning that it will not try to iterate unbounded fields. Below we see that, since `maxbits` on H is 5 (H0-H4), we get 21 unique bits total, just as above.

In [31]:
bits = [bit for bit in a]
len(bits)

21

In [32]:
all([bit in a for bit in a])

True

In [33]:
b = reduce(lambda p,q: p+q, a) ; b

(S:  1000000 H:  11111 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  1000000 H:  1000 X:  10000 x:   1 r:   1 aA:   1) [Order:  10 aA:   1] (S:  10)

In [34]:
a in b

False

In [35]:
b in a

True

In [36]:
b.to_smarts()

'[#6;H0,H1,H2,H3,H4;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#1:3]'

In [37]:
a.to_smarts()

'[#6;X4;x0;!r;A:1]-;!@[#6;H3;X4;x0;!r;A:2]-;!@[#1:3]'

We see `b` is a subset of `a` since the bit enumeration uses a cutoff in the H field, since it is unbounded in `a`

In [38]:
for i, primitive in enumerate(a.to_primitives()):
    print(f"{i+1:2d}", primitive.to_smarts())

### Here is an example of how to use Tanimoto scores to see similarity between parameters

In [39]:
def tanimoto(x, y):
    x = x.copy()
    x.align_to(y)
    A = x.bits()
    B = y.bits()
    C = (x & y).bits()
    return A, B, C, C / (A + B - C)

In [40]:
tanimoto(x,y)

(17, 17, 17, 1.0)

In [41]:
tanimoto(xy,x)

(24, 17, 17, 0.7083333333333334)

In [42]:
tanimoto(xy,a)

(24, inf, 17, 0.0)

In [43]:
tanimoto(xy,b)

(24, 21, 17, 0.6071428571428571)